In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


In [4]:
## import numpy as np
import pandas as pd
import numpy as np

import yfinance as yf
import bokeh

# from threading import Thread
# import time
from functools import partial

from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, HoverTool,CrosshairTool, Div,Paragraph, Spinner, Dropdown #Span, MultiChoice,
from bokeh.plotting import ColumnDataSource, figure, output_file, show, curdoc
from bokeh.models import Panel, Tabs, DatePicker, Select, TextInput #NumeralTickFormatter, PreText, Quadratic, 
# import bokeh.plotting as bk
from bokeh.palettes import d3

from datetime import datetime, date, timedelta

from bokeh.io import show, save, output_notebook, reset_output
from bokeh.models import CustomJS, RangeSlider #, DataRange1d, Div

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

# from tornado import gen
import locale
from functools import partial

#[X] Stock Splits
#[X] Organize by tab
#[X] Create different source for each tab
#[X] Last day always equals current value 
# [] Function string to dates
# [] consistent number scheme for tabs and variables 
# [X] index fund includes only trading days
# [X] X axis not updating
# [X] Div sign 
# [X] source.update(callback) so other tabs don't change 
# [X] Minimum dates based on tickers
# [] Pop-up for start dates before IPO
# [X] Three different plot windows
# [X] Min date between the two choices for each tab
# [] current investment value in div box
    
output_notebook()
locale.setlocale(locale.LC_ALL, '')
def modify_doc(doc):
           
    ticker_symbols = {
        'DJI': '^DJI',
        'S&P 500': '^GSPC'
    }
    
    def yf_fund(ticker, start_date, end_date, principal):
        df_yf_fund = pd.DataFrame()
        
        
        if ticker in [i for i in ticker_symbols.keys()]:
            ticker_label = ticker
#             print(f'Label: {ticker_label}')
            ticker = ticker_symbols[ticker]
#             print(f'Ticker: {ticker}')
        else: 
            ticker_label = ticker
        
        yf_fund_ticker = yf.Ticker(ticker)
        end_date2 = end_date + timedelta(1)
        end_date2 = str(end_date2)
        start_date = str(start_date)
        
        df_yf_fund = yf_fund_ticker.history(start=start_date, end=end_date2)
        yf_fund_cost_basis = df_yf_fund.iloc[0, 3]
        no_shares = principal/yf_fund_cost_basis
        df_yf_fund['Position'] = df_yf_fund['Close'] * no_shares
        for i in df_yf_fund[df_yf_fund['Stock Splits'] > 0].index.values:
            df_yf_fund.at[i, 'Position'] *= df_yf_fund.at[i, 'Stock Splits']
            
        df_yf_fund['legend'] = np.zeros_like(df_yf_fund['Position'])
        df_yf_fund['legend'] = ticker_label            
        df_yf_fund.columns = [f'Stock {i}' for i in df_yf_fund.columns]
        #print(f'{ticker} {df_yf_fund.index}')
        return df_yf_fund, yf_fund_cost_basis
    
    def managed_fund(principal, current_value, df_yf_fund):
        df_managed_fund = pd.DataFrame()
        start_date = pd.to_datetime(df_yf_fund.index[0]) 
        end_date = pd.to_datetime(df_yf_fund.index[-1]) 
        period = (end_date - start_date).days
        period_years = period/365.25
        rate = ((current_value/principal)**(1/period_years)) - 1
        
        df_managed_fund = pd.DataFrame()
        df_managed_fund['Date'] = [(start_date + timedelta(i)) for i in range(period + 1)]
        df_managed_fund['Date'] = pd.to_datetime(df_managed_fund['Date'])
        df_managed_fund['Position'] = [principal * (1 + rate) ** (i/365.25) for i in range(period + 1)]
        df_managed_fund = df_managed_fund[df_managed_fund['Date'].isin(df_yf_fund.index.values)]
        df_managed_fund = df_managed_fund.set_index('Date')
        df_managed_fund['legend'] = np.zeros_like(df_managed_fund['Position'])
        df_managed_fund['legend'] = 'Managed Fund'
        df_managed_fund.columns = [f'Managed {i}' for i in df_managed_fund.columns]
        #print(f'Manged fund: {df_managed_fund.index}')
        return df_managed_fund, rate
        
    def create_source(df_fund1, df_fund2):
        df_source = pd.DataFrame()
        df_fund1.index = pd.to_datetime(df_fund1.index)
        df_fund2.index = pd.to_datetime(df_fund2.index)
 
        legend1 = next(i for i in df_fund1.columns if 'legend' in i)
        legend2 = next(i for i in df_fund2.columns if 'legend' in i)
        
        df_fund1 = df_fund1.rename(columns={legend1:'legend1'})
        df_fund2 = df_fund2.rename(columns={legend2:'legend2'})
        
        col1 = next(i for i in df_fund1.columns if 'Position' in i)
        col2 = next(i for i in df_fund2.columns if 'Position' in i)
        
        df_source = df_fund1.join(df_fund2, how='inner', rsuffix='_2')
        
        df_source['Difference'] = df_fund1[col1] - df_fund2[col2]
        #print(f'df_source: {df_source.columns}')
        return df_source
    
    def make_plot(df_source, title):
        source = ColumnDataSource(df_source)
        position_col = [i for i in df_source.columns if 'Position' in i]
        #print(position_col)
        line1 = position_col[0]
        line2 = position_col[1]
        
        labels = [x.strip() for x in title.split('vs.')]
        label1 = labels[1]
        label2 = labels[0]
        
        TOOLTIPS = [
                ('Date', '@Date{%F}'),
                (f'{label1}', f'@{{{line1}}}{{$0,0}}'),        
                (f'{label2}', f'@{{{line2}}}{{$0,0}}'),
                ('Difference', '@Difference{$0,0}'),                
                ]
          
        plot = figure(width_policy = 'fit', height_policy = 'fit', x_axis_type='datetime', title = title)
#        print((df_source.index[0], df_source.index[-1]))
        plot.line('Date', line1, source = source, legend_field = 'legend1', color = '#1f77b4', line_width = 3)
        plot.line('Date', line2, source = source, legend_field = 'legend2', color = d3['Category10'][10][1], line_width = 3)
        plot.add_tools(CrosshairTool())
        plot.add_tools(HoverTool(tooltips = TOOLTIPS, formatters={'@Date': 'datetime'}))
        plot.legend.location = 'top_left'
        plot.legend.click_policy = 'hide'
        plot.xaxis.axis_label = 'Date'
        plot.yaxis.axis_label = 'USD ($)'

        return plot, source        

    def div_text(df_source, cost_basis, investment_type):
        legend1 = next(i for i in df_source.columns if 'legend' in i)
        ticker1 = df_source[legend1][0]
        legend2 = next(i for i in df_source.columns if 'legend' in i) 
        ticker2 = df_source[legend2][0]
        
        position_col = next(i for i in df_source.columns if 'Position' in i)
        current_value = df_source[position_col][-1]
        principal = df_source[position_col][0]
        growth = (current_value - principal)/principal 
        verb = 'appreciated' if growth > 0 else 'depreciated'
        difference = df_source['Difference'][-1]
        pos_sign = '+' if float(difference) > 0 else ''
        difference = locale.currency(difference, grouping = True)
        cost_basis = locale.currency(cost_basis, grouping = True)
        text =(
              f'Your {investment_type} {verb} {growth: .0%}.'
              f'<br>If you invested in {ticker1}, you would have {pos_sign}{difference}.'
              f'<br>Cost basis for {ticker2}: {cost_basis}/share'
              )
        
        return text
    
    # Callbacks all 
#     def update(attr, old, new, i):

    def update(attr, old, new, i):
        start_date = pd.to_datetime(start_date_picker[i].value).date()
        end_date = pd.to_datetime(end_date_picker[i].value).date()
        principal = principal_spinner[i].value
        current_value = current_value_spinner[i].value
#         ticker = ticker_symbols[fund_1[1].value]
        min_date = find_min_date(i)

    #         print(min_date)
        start_date_picker[i].min_date = min_date
        
        print(f'date test:{start_date < min_date}')
        if start_date < min_date:
            start_date_picker[i].value = min_date
            start_date = min_date
        
        if i == 1:
            df_fund_1[1], index_cost_basis = yf_fund(fund_1[1].value, start_date, end_date, principal)

            df_fund_2[1], rate = managed_fund(principal, current_value, df_fund_1[1])
            df_source[1] = create_source(df_fund_1[1], df_fund_2[1])

            new_source = ColumnDataSource(df_source[1])
            source[i].data.update(new_source.data)
            div[i].text = div_text(df_source[1], index_cost_basis, 'mutual fund')   

            print(df_source[1].columns)
    #         min_date1 = yf.Ticker(ticker_symbols[ticker]).history(period='max').head(1).index[0].date()
    #         min_date2 = yf.Ticker(stock_input.value).history(period='max').head(1).index[0].date()
    #         min_date3 = yf.Ticker(fund_2[3].value).history(period='max').head(1).index[0].date()
    #         min_date = max(min_date1, min_date2, min_date3) 
#             min_date = find_min_date(i)

#     #         print(min_date)
#             start_date_picker[i].min_date = min_date
        
        # Tab 2
#         if i == 2:
            
#             df_fund_1[2], stock_cost_basis = yf_fund(fund_1[2].value, start_date, end_date, principal)
#             print('Tab 2')
# #             print(f'Start date: {start_date}')
# #             print(f'End date: {end_date}')
            
# #             df_fund_2[2] = df_fund_1[1]
#             df_fund_2[2], index_cost_basis = yf_fund(fund_2[2].value, start_date, end_date, principal)
#             print(f'df_fund_1[2] len: {len(df_fund_1[2])}')
#             print(f'df_fund_2[2] len: {len(df_fund_2[2])}')
# #             print(f'df_fund_1[2].columns: {df_fund_1[2].columns}')
# #             print(f'df_fund_2[2].columns: {df_fund_2[2].columns}')
        
        
#             df_source[2] = create_source(df_fund_1[2], df_fund_2[2])
# #             print(df_source[2].columns)
            

#             new_source2 = ColumnDataSource(df_source[2])
# #             print(new_source2.data)
#             source2.data.update(new_source2.data)

#             current_value_stock = source2.data['Stock Position'][-1]
#             div2.text = div_text(df_source[2], stock_cost_basis, f'{fund_1[2].value} investment')
        
        # Tab 3
        else:
            
            df_fund_1[i], stock_cost_basis = yf_fund(fund_1[i].value, start_date, end_date, principal)
#             df_fund_2[3] = df_fund_2[2]
            df_fund_2[i], stock2_cost_basis = yf_fund(fund_2[i].value, start_date, end_date, principal)
            df_source[i] = create_source(df_fund_1[i], df_fund_2[i])
#             print(df_source[i].columns)
            
            new_source = ColumnDataSource(df_source[i])
            source[i].data.update(new_source.data)        

#             current_value_stock2 = source3.data['Stock Position'][-1]
            div[i].text = div_text(df_source[3], stock_cost_basis, f'{fund_1[3].value} investment')
            

    
#     callback_holder = PreText(text='', css_classes=['hidden'], visible=False)
    
    def find_min_date(i):
#         stock = {
#                 1: ticker_symbols[ticker], 
#                 2: stock_input.value,
#                 3: fund_2[3].value
#                 }
        
#         min_date = yf.Ticker(stock[i]).history(period='max').head(1).index[0].date()
        
        if i == 1:
            ticker = ticker_symbols[fund_1[i].value]
            print(ticker)
            min_date = yf.Ticker(ticker).history(period='max').head(1).index[0].date()
        else:
            min_date_top = yf.Ticker(fund_1[i].value).history(period='max').head(1).index[0].date()
            
#             list_test = [i for i in ticker_symbols.keys()]
#             print(list_test)
            if fund_2[i].value in [i for i in ticker_symbols.keys()]:
                ticker = ticker_symbols[fund_2[i].value]
                min_date_bottom = yf.Ticker(ticker).history(period='max').head(1).index[0].date()
            else:
                min_date_bottom = yf.Ticker(fund_2[i].value).history(period='max').head(1).index[0].date()
            
            min_date = max(min_date_top, min_date_bottom)

        return min_date
    
#     def check_date(attrname, old, new):
#         min_date = yf.Ticker(ticker_symbols[ticker]).history(period='max').head(1).index[0].date()
#         if old < min_date:
#             callback_holder.text = "Alert!!! Start date is before stock went public"
         
    
#     def min_date_function(attrname, old, new):
#         global something
#         if something is not None:
#             tab_managed.children.remove(tab_managed.children[5])
        
#         min_date1 = yf.Ticker(ticker_symbols[ticker]).history(period='max').head(1).index[0].date()
#         min_date2 = yf.Ticker(stock_input.value).history(period='max').head(1).index[0].date()
#         min_date3 = yf.Ticker(fund_2[3].value).history(period='max').head(1).index[0].date()
#         min_date = min(min_date1, min_date2, min_date3)
        
#         start_date_picker = DatePicker(title = 'Start Date', value = start_date, min_date = min_date, max_date = max_date)#, min_date="2019-08-01", max_date="2019-10-30")
#         inputs = column(principal_spinner, current_value_spinner, ticker_select, start_date_picker, end_date_picker, div)
#         tab_managed = Panel(child = row(plot1, inputs), title = 'Managed Fund vs Index Fund')
       
#         layout = Tabs(tabs=[tab_managed, tab_stock, tab_stock2])
# #         doc.add_root(layout)
#         something = 1
        
        
    
    #WIDGETS
    
    principal = 1000.0
    current_value = 3000.0
    ticker = 'S&P 500'
    start_date = date(2021, 5, 3)
    end_date = date(2021, 5, 7)
#     min_date = yf.Ticker(ticker_symbols[ticker]).history(period='max').head(1).index[0].date()
#     max_date = yf.Ticker(ticker_symbols[ticker]).history(period='max').tail(1).index[0].date()
    min_date = yf.Ticker(ticker_symbols[ticker]).history(period='max').index[0].date()
    max_date = yf.Ticker(ticker_symbols[ticker]).history(period='max').index[-1].date()
    
    
    #print(min_date)
    start_date_picker = {}
    end_date_picker = {}
    principal_spinner = {}
    current_value_spinner = {}
    fund_1 = {}
    fund_2 = {}
    
#     ticker_select = Select(title='Index Fund', value = 'S&P 500', options = ['DJI', 'S&P 500'])
#     stock_input = TextInput(value="AMZN", title="Stock Ticker Symbol")
#     fund_2[3] = TextInput(value="GOOG", title="Stock 2 Ticker Symbol")
    fund_1[1] = Select(title='Index Fund', value = 'S&P 500', options = ['DJI', 'S&P 500'])
    fund_2[1] = None
    
    fund_1[2] = TextInput(value="AMZN", title="Stock Ticker Symbol")
    fund_2[2] = Select(title='Index Fund', value = 'S&P 500', options = ['DJI', 'S&P 500'])
    
    fund_1[3] = TextInput(value="AMZN", title="Stock Ticker Symbol")
    fund_2[3] = TextInput(value="GOOG", title="Stock 2 Ticker Symbol")
    
    for i in [1,2,3]:
        start_date_picker[i] = DatePicker(title = 'Start Date', value = start_date, min_date = find_min_date(i), 
                                          max_date = max_date)#, min_date="2019-08-01", max_date="2019-10-30")
        end_date_picker[i] = DatePicker(title = 'End Date', value = end_date, min_date = find_min_date(i), 
                                        max_date = max_date) 
        principal_spinner[i] = Spinner(value=principal, step=1, title='Principal')
        current_value_spinner[i] = Spinner(value=current_value, step=1, title='Current Value')



    df_fund_1 = {}
    df_fund_2 = {}
    df_source = {}
    source = {}
    div = {}
    
    # Tab 1
    #Get data
        
    df_fund_1[1], index_cost_basis = yf_fund(fund_1[1].value, start_date, end_date, principal)
#     print(fund_1[1])

    df_fund_2[1], rate = managed_fund(principal, current_value, df_fund_1[1])

    df_source[1] = create_source(df_fund_1[1], df_fund_2[1])
    
    #Set-up Plots
    
    plot1, source[1] = make_plot(df_source[1], 'Managed Fund vs. Index Fund')

    div[1] = Div(text = div_text(df_source[1], index_cost_basis, 'mutual fund'), 
              sizing_mode = 'stretch_width', height=100)    
        
    # Layout
    
    inputs = column(principal_spinner[1], current_value_spinner[1], fund_1[1], start_date_picker[1], 
                    end_date_picker[1], div[1])
#     global tab_managed
    tab_managed = Panel(child = row(plot1, inputs), title = 'Managed Fund vs Index Fund')
  
    # Tab2
    
    # Data
#     df_fund_1[2] = df_fund_1[1]
    df_fund_1[2], index_cost_basis = yf_fund(fund_1[2].value, start_date, end_date, principal)
    df_fund_2[2], stock_cost_basis = yf_fund(fund_2[2].value, start_date, end_date, principal)
    
    df_source[2] = create_source(df_fund_1[2], df_fund_2[2])
    print(f'df_fund_1[2].columns: {df_fund_1[2].columns}')
    print(f'df_fund_2[2].columns: {df_fund_2[2].columns}')
    # Plots
    
    plot2, source[2] = make_plot(df_source[2], 'Stock vs. Index Fund')
#     print(f'source2 columns: {df_source[2].columns}')
    
    current_value_stock = df_source[2]['Stock Position'][-1]
    div[2] = Div(text = div_text(df_source[2], stock_cost_basis, f'{fund_1[2].value} investment'), 
               sizing_mode = 'stretch_width', height=100)
    # Layout
    
    inputs_stock = column(principal_spinner[2], fund_1[2], fund_2[2], start_date_picker[2], end_date_picker[2], div[2])
    tab_stock = Panel(child = row(plot2, inputs_stock), title = 'Stock vs Index Fund')
    
    # Tab3
    
    # Data
    
    df_fund_1[3], stock2_cost_basis = yf_fund(fund_2[3].value, start_date, end_date, principal)
    df_fund_2[3] = df_fund_2[2]
    df_source[3] = create_source(df_fund_1[3], df_fund_2[3])
    
    # Plot
    
    plot3, source[3] = make_plot(df_source[3], 'Stock vs. Stock 2')
    
    current_value_stock2 = df_source[3][f'Stock Position'][-1]
    div[3] = Div(text = div_text(df_source[3], stock_cost_basis, f'{fund_1[3].value} investment'), 
               sizing_mode = 'stretch_width', height=100)
    
    # Layout
    
    inputs_stock2 = column(principal_spinner[3], fund_1[3], fund_2[3], start_date_picker[3], end_date_picker[3], div[3])        
    tab_stock2 = Panel(child = row(plot3, inputs_stock2), title = 'Stock vs Stock 2')

    layout = Tabs(tabs=[tab_managed, tab_stock, tab_stock2])
    
 
    #start_date_picker.on_change('value', check_date)
    
    for i in [1, 2, 3]:
        test = start_date_picker[i]
        start_date_picker[i].on_change('value', partial(update, i = i))
        end_date_picker[i].on_change('value', partial(update, i = i))
        principal_spinner[i].on_change('value', partial(update, i = i))
        current_value_spinner[i].on_change('value', partial(update, i = i))
        
        fund_1[i].on_change('value', partial(update, i = i))    
        if i != 1:
            fund_2[i].on_change('value', partial(update, i = i))
        
#         ticker_select.on_change('value', partial(update, i = i))    
#         stock_input.on_change('value', partial(update, i = i))  
#         fund_2[3].on_change('value', partial(update, i = i))  
        
#     start_date_picker[1].on_change('value', partial(update, i = 1))

#     ticker_select.on_change('value', partial(update, i = None))    
#     stock_input.on_change('value', partial(update, i = None))  
#     fund_2[3].on_change('value', partial(update, i = None))  
    
#     ticker_select.on_change('value', min_date_function)    
#     stock_input.on_change('value', min_date_function)
#     fund_2[3].on_change('value', min_date_function)
    
    #Set-up layout 


    doc.add_root(layout)
    
handler = FunctionHandler(modify_doc)
app = Application(handler)
show(app, notebook_url="localhost:8889")

Loading BokehJS ...

^GSPC
^GSPC
df_fund_1[2].columns: Index(['Stock Open', 'Stock High', 'Stock Low', 'Stock Close', 'Stock Volume',
       'Stock Dividends', 'Stock Stock Splits', 'Stock Position',
       'Stock legend'],
      dtype='object')
df_fund_2[2].columns: Index(['Stock Open', 'Stock High', 'Stock Low', 'Stock Close', 'Stock Volume',
       'Stock Dividends', 'Stock Stock Splits', 'Stock Position',
       'Stock legend'],
      dtype='object')
date test:False


In [1]:
# BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('Date', 6034), ('Difference', 6034), ('Managed Position', 5), ('Stock Close', 6034), ('Stock Close_2', 6034), ('Stock Dividends', 6034), ('Stock Dividends_2', 6034), ('Stock High', 6034), ('Stock High_2', 6034), ('Stock Low', 6034), ('Stock Low_2', 6034), ('Stock Open', 6034), ('Stock Open_2', 6034), ('Stock Position', 6034), ('Stock Position_2', 6034), ('Stock Stock Splits', 6034), ('Stock Stock Splits_2', 6034), ('Stock Volume', 6034), ('Stock Volume_2', 6034), ('legend1', 6034), ('legend2', 6034)